In [1]:
import pennylane as qml
import numpy as np

##### SWAP Test Demo 


In [2]:

dev = qml.device('default.qubit',wires=3,shots=500)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(0)
    qml.ctrl(qml.SWAP([1,2]),control=[0])
    qml.Hadamard(0)
    return qml.probs(0)
circuit()

tensor([1., 0.], requires_grad=True)

##### Amplitude Embedding demo

In [14]:
dev_amp = qml.device('default.qubit',wires=2)

@qml.qnode(dev_amp)
def amp_circuit(feature:list):
    qml.AmplitudeEmbedding(feature,wires=range(2))
    return qml.state()

amp_circuit([1/2,1/2,1/2,1/2])

tensor([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j], requires_grad=True)